The goal is to use topic modeling to investigate the relationship between some philosophical texts. In a previous step, I scraped them up from the Gutenberg project and stored them locally as text files.

To start building the topic model - load in some libraries, including numpy and sklearn.

In [1]:
import glob, sys, string
import numpy as np
from sklearn import feature_extraction, decomposition

Define some new functions.

In [2]:
def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
                print("Topic #%d:" % topic_idx)
                print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

def topic_modeling_prep(filenames, n_features):
        vectorizer = feature_extraction.text.CountVectorizer(input='filename', max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
        tf_matrix = vectorizer.fit_transform(filenames)
        tf_array = tf_matrix.toarray()
        return vectorizer, tf_matrix, tf_array

def calc_lda(tf_array, n_topics=10,max_iter=20):
        lda = decomposition.LatentDirichletAllocation(n_topics=n_topics, max_iter=max_iter,
                                learning_method='online',
                                learning_offset=50.)
        doctopic = lda.fit_transform(tf_array)
        return doctopic, lda

def assign_topic(text_names,doctopic,num_topics):
        doctopic_grouped = np.zeros((text_names.shape[0], num_topics))
        for i, name in enumerate(text_names):
                doctopic_grouped[i, :] = np.mean(doctopic[text_names == name, :], axis=0)
        doc_topic_assign = np.argmax(doctopic_grouped, axis=1)
        return doc_topic_assign

def summarize_results(doc_topic_assign, lda, vectorizer, filenames):
        summary = read_summary()
        bookids = [f[5:-4] for f in filenames]

        # for each topic, print out words and documents.
        feature_names = vectorizer.get_feature_names()
        n_top_words = 5
        for topic_idx, topic in enumerate(lda.components_):
                print "Topic #%d:" % topic_idx,
                print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

                # which docs are placed in this topic?
                ids = np.where(doc_topic_assign==topic_idx)[0].tolist()
                print("\n".join([summary[(bookids[i],'title')] + " by " + summary[(bookids[i],'author')]  for i in ids]))
                print("\n")

def read_summary():
        # read in summary file with title/author/year info
        summary = {}
        with open('data/summary.txt','r') as f1:
                for line in f1:
                        cols                            = line.split('\t')
                        summary[(str(cols[0]),'title')] = str(cols[1])
                        summary[(str(cols[0]),'author')]= str(cols[2])
                        summary[(str(cols[0]),'year')]  = str(cols[3]).rstrip()
        return summary


Now let's build that model! This step takes a few minutes.

In [3]:
# constants
num_topics = 5
num_features = 1000

# read input files
filenames = glob.glob("data/[0-9]*.txt")
text_names = np.asarray(filenames)

# create model
vectorizer, tf_matrix, tf_array = topic_modeling_prep(filenames,num_features)
doctopic, lda = calc_lda(tf_array,num_topics)
doc_topic_assign = assign_topic(text_names,doctopic,num_topics)

# describe results
summarize_results(doc_topic_assign, lda, vectorizer,filenames)


Topic #0: greek small letter laws cause
Review of the Work of Mr John Stuart Mill Entitled, 'Examination of Sir William Hamilton's Philosophy.' by George Grote
A System of Logic: Ratiocinative and Inductive by John Stuart Mill
A System Of Logic, Ratiocinative And Inductive by John Stuart Mill
Analysis of Mr. Mill's System of Logic by William Stebbing
Our Knowledge of the External World as a Field for Scientific Method in Philosophy by Bertrand Russell
Poetics by Aristotle
Logic, Inductive and Deductive by William Minto
Symbolic Logic by Lewis Carroll
A System Of Logic, Ratiocinative And Inductive (Vol. 1 of 2) by John Stuart Mill
Logic by Carveth Read
The Categories by Aristotle
Ontology or the Theory of Being by Peter Coffey
A System of Logic: Ratiocinative and Inductive by John Stuart Mill
The Game of Logic by Lewis Carroll


Topic #1: world philosophy experience reality footnote
A History of Indian Philosophy, Vol. 1 by Surendranath Dasgupta
Bergson and His Philosophy by J. Alexande

That's pretty cool and some of the groups make sense. Is there a way to visualize an LDA model?

In [4]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [5]:
pyLDAvis.sklearn.prepare(lda,tf_matrix, vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      25.730588        1       1 -0.086943  0.044948
2      22.168360        1       2  0.054870 -0.010941
1      21.454346        1       3 -0.005688 -0.070162
3      15.631668        1       4  0.067296  0.053464
4      15.015037        1       5 -0.029535 -0.017308, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
544   Default  4951.000000        letter  4951.000000  30.0000  30.0000
422   Default  5979.000000         greek  5979.000000  29.0000  29.0000
859   Default  5747.000000         small  5747.000000  28.0000  28.0000
420   Default  2500.000000    government  2500.000000  27.0000  27.0000
460   Default  5666.000000          idea  5666.000000  26.0000  26.0000
386   Default  2641.000000      footnote  2641.000000  25.0000  25.0000
463   Default  4972.000000         ideas  4972.000000  24.0000  24.0000
415   Default  6561.000000           god  6561.000000  23.0000  23.0000
862   Default  2084.000000      socrates  2084.000000  22.0000  22.0000
669   Default  3402.000000        people  3402.000000  21.0000  21.0000
600   Default  4392.000000         moral  4392.000000  20.0000  20.0000
988   Default  7683.000000         world  7683.000000  19.0000  19.0000
755   Default  2500.000000  propositions  2500.000000  18.0000  18.0000
754   Default  3134.000000   proposition  3134.000000  17.0000  17.0000
860   Default  2922.000000        social  2922.000000  16.0000  16.0000
931   Default  1653.000000          thou  1653.000000  15.0000  15.0000
489   Default  1687.000000     induction  1687.000000  14.0000  14.0000
128   Default  6891.000000         cause  6891.000000  13.0000  13.0000
761   Default  1634.000000        public  1634.000000  12.0000  12.0000
115   Default   815.000000         burke   815.000000  11.0000  11.0000
630   Default  6073.000000        object  6073.000000  10.0000  10.0000
779   Default  3811.000000       reality  3811.000000   9.0000   9.0000
108   Default  4741.000000          body  4741.000000   8.0000   8.0000
320   Default  1676.000000           est  1676.000000   7.0000   7.0000
976   Default  1859.000000           war  1859.000000   6.0000   6.0000
687   Default  6189.000000    philosophy  6189.000000   5.0000   5.0000
105   Default  1047.000000       bergson  1047.000000   4.0000   4.0000
468   Default  1858.000000           iii  1858.000000   3.0000   3.0000
748   Default   710.000000          prop   710.000000   2.0000   2.0000
129   Default  4646.000000        causes  4646.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
172    Topic5   676.398596      conceive  1524.042268   1.0838  -5.6207
460    Topic5  2480.501629          idea  5666.692042   1.0700  -4.3213
179    Topic5   704.625837    concerning  1602.727418   1.0743  -5.5798
747    Topic5   927.620539         proof  2159.696791   1.0510  -5.3049
694    Topic5   955.751561      pleasure  2250.088696   1.0399  -5.2750
472    Topic5   732.901347   imagination  1723.695406   1.0409  -5.5405
108    Topic5  1724.807731          body  4741.412211   0.8849  -4.6846
630    Topic5  1929.202425        object  6073.559706   0.7493  -4.5726
632    Topic5  1497.369924       objects  4596.396223   0.7746  -4.8260
415    Topic5  1825.139179           god  6561.879273   0.6165  -4.6281
356    Topic5   917.287182      external  2512.413434   0.8885  -5.3161
128    Topic5  1786.758504         cause  6891.884128   0.5462  -4.6493
340    Topic5  1527.190569     existence  5663.407066   0.5855  -4.8063
564    Topic5   941.387738          love  2779.639206   0.8134  -5.2901
317    Topic5   712.871098       essence  1776.395522   0.9831  -5.5682
457    Topic5  1495.141625         human  6613.360321   0.4092  -4.8275
249    Topic5   801.666093        desire  2253.8604